In [ ]:
"""
Manuscript figures
"""

import os, sys
import geopandas as gpd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.patches as mpatches

from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize
from sklearn.preprocessing import MinMaxScaler
from mapclassify import Quantiles

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')
icsdir = os.path.join(maindir, 'ics209-plus-fired/data/')

print("Ready !")

In [ ]:
# load fire perimeter data
fp = os.path.join(projdir,'data/spatial/mod/nifc-ics_2018_to_2023-aspen.gpkg')
fires = gpd.read_file(fp)
fires.columns

In [ ]:
# load the frp grid
fp = os.path.join(projdir,f'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats.gpkg')
grid = gpd.read_file(fp)
grid.columns

In [ ]:
fp = os.path.join(projdir,'data/spatial/raw/boundaries/na_cec_eco_l3_west.gpkg')
ecol3 = gpd.read_file(fp)
ecol3 = ecol3[['NA_L3NAME','NA_L3CODE','geometry']].rename(columns={'NA_L3NAME': 'na_l3name', 'NA_L3CODE': 'na_l3code'})
ecol3.columns

In [ ]:
# Figure 1A. Map of the Southern Rockies

In [ ]:
# Generate centroids
fires_ = fires[fires['NIFC_ID'].isin(grid['Fire_ID'].unique())]
fires_['NIFC_ACRES'] = pd.to_numeric(fires_['NIFC_ACRES'], errors='coerce')
fires_['FINAL_KM2'] = fires_['NIFC_ACRES'] * 0.00404686
fires_['FINAL_HA'] = fires_['NIFC_ACRES']* 0.404686
print(fires_['FINAL_HA'].describe())

# Get the centroid of fires
centroid = fires_.copy()
centroid['geometry'] = centroid.geometry.centroid
centroid['aspen_size'] = centroid['pct_cover'] * 5  

# # do some scaling (area weighted aspen cover)
# scaler = MinMaxScaler(feature_range=(0, 100))  # Normalize to 0–100
# centroid['pct_cover_wt'] = centroid['pct_cover'] * centroid['FINAL_HA']  # Weighted by fire size
# centroid['pct_cover_wt_nm'] = scaler.fit_transform(centroid[['pct_cover_wt']])

scaler = MinMaxScaler(feature_range=(10, 100))  # Marker sizes between 10 and 200
centroid['fire_size'] = scaler.fit_transform(centroid[['FINAL_HA']]) * 3

# subset to SRM
centroid_srm = centroid[centroid['na_l3name'] == 'Southern Rockies']
ecol3_srm = ecol3[ecol3['na_l3name'] == 'Southern Rockies']

# Ensure the same projection
centroid_srm = centroid_srm.to_crs(4326)
ecol3_srm = ecol3_srm.to_crs(4326)

# plot it
fig, ax = plt.subplots(figsize=(6, 6))

ecol3_srm.plot(ax=ax, edgecolor='black', linewidth=0.8, color='none')

centroid_srm.plot(
    ax=ax, 
    markersize=centroid_srm['fire_size'],  # Use normalized marker size
    column='pct_cover', cmap='viridis', 
    legend=False, alpha=0.6)

# Add colorbar for aspen percent cover
norm = mcolors.Normalize(vmin=centroid_srm['pct_cover'].min(), vmax=centroid_srm['pct_cover'].max())
sm = ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, shrink=0.7)
cbar.set_label('Aspen proportion (%)')

# Create a custom legend for fire size
# Create legend sizes for fire size (in hectares)
legend_sizes_ha = [1000, 50000, 100000]  # Example fire sizes in hectares
legend_scaled_sizes = scaler.transform([[size] for size in legend_sizes_ha]).flatten() * 3
legend_labels = [f'-={int(size / 1000)}k' for size in legend_sizes_ha]

# Create legend handles using the correctly scaled sizes
legend_handles = [
    plt.scatter([], [], s=size, color='gray', alpha=0.6, label=label)
    for size, label in zip(legend_scaled_sizes, legend_labels)
]

# Add the legend to the map
ax.legend(
    handles=legend_handles, loc='upper left', 
    title='Fire Size (ha)', frameon=False, fontsize=8, title_fontsize=9, 
    handletextpad=0.8,  # Space between markers and labels
    labelspacing=1.2    # Vertical space between legend items
)

# Labels and saving
plt.xlabel('Longitude', size=10)
plt.ylabel('Latitude', size=10)

# Save the map
plt.savefig(os.path.join(projdir, 'figures/Figure1_Aspen-Fires_NIFC-SRM.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# plot the annual area burned
burned_area = fires_.groupby('START_YEAR')['FINAL_HA'].sum().reset_index()

# set up the axis formatter
def format_thousands(x, pos):
    """Format y-axis labels to display values in 'k' for thousands."""
    return f'{int(x / 1000)}k'

formatter = FuncFormatter(format_thousands)

# Plot the results
plt.figure(figsize=(4, 2.5))
plt.bar(burned_area['START_YEAR'], burned_area['FINAL_HA'], color='grey', edgecolor='black')
plt.title('Annual Area Burned (2018-2023)', fontsize=11)
plt.xlabel('Year', fontsize=10)
plt.ylabel('Area Burned (ha)', fontsize=9)
plt.xticks(burned_area['START_YEAR'], rotation=45, size=8)

# Apply custom formatting to y-axis
plt.gca().yaxis.set_major_formatter(formatter)
# plt.ylim(10000, None)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.savefig(os.path.join(projdir, 'figures/Figure1b_Aspen-Fires_NIFC-SRM_annualBurnedArea.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# plot the density distribution of fire size
plt.figure(figsize=(4, 2.5))
sns.kdeplot(fires_['FINAL_HA'], fill=True, color='grey', alpha=1)
plt.title('Distribution of Fire Sizes', fontsize=11)
plt.xlabel('Fire Size (ha)', fontsize=9)
plt.ylabel('Density', fontsize=9)

plt.gca().xaxis.set_major_formatter(formatter)

plt.grid(alpha=0.3)
plt.tight_layout()

plt.savefig(os.path.join(projdir, 'figures/Figure1b_Aspen-Fires_NIFC-SRM_fireSize.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Figure 2. Conceptual workflow for VIIRS AFD.

In [ ]:
# Fire 2A. FRP detections by censor
# load the AFD points
afd = os.path.join(projdir,f'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_aspenfires.gpkg')
afd = gpd.read_file(afd)

# Figure 2B. Pixel ground area
pix = os.path.join(projdir,f'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_nd.gpkg')
pix = gpd.read_file(pix)

pix.columns

In [ ]:
# subset data to example fire:
afd_fire = afd[afd['Fire_Name'] == 'WILLIAMS FORK']
pix_fire = pix[pix['Fire_Name'] == 'WILLIAMS FORK']
fire = grid[grid['Fire_Name'] == 'WILLIAMS FORK']

afd_fire = afd_fire.to_crs(4326)
pix_fire = pix_fire.to_crs(4326)
fire = fire.to_crs(4326)

# plot them
fig, axes = plt.subplots(1, 3, figsize=(10, 7), constrained_layout=True)

# Panel 1: Active fire detection points
afd_fire.plot(
    markersize=8,
    column='satellite', 
    legend=True,
    alpha=0.6,
    ax=axes[0]
)
# axes[0].set_title('Active Fire Detections', fontsize=11)
axes[0].axis('off')

# Panel 2: Pixel ground areas
pix_fire.plot(
    column='satellite', 
    legend=False, 
    alpha=0.6,
    ax=axes[1]
)
# axes[1].set_title('Pixel Ground Areas', fontsize=11)
axes[1].axis('off')  # Remove borders and axes for a clean look

# Panel 3: Aggregated FRP
n_bins = 21
quantiles = Quantiles(fire['frp_max'], k=n_bins)
fire['frp_max_bins'] = quantiles.yb

fire.plot(
    column='frp_max_bins',
    cmap='inferno',
    linewidth=0,
    alpha=0.6,
    ax=axes[2]
)
# axes[2].set_title('Aggregated FRP', fontsize=11)
axes[2].axis('off')  # Remove borders and axes for a clean look

# Add a continuous colorbar to the third panel
norm = Normalize(vmin=fire['frp_max'].min(), vmax=fire['frp_max'].max())
sm = plt.cm.ScalarMappable(cmap='inferno', norm=norm)
sm._A = []  # Dummy array for ScalarMappable
cbar = fig.colorbar(sm, ax=axes[2], orientation='vertical', fraction=0.04, pad=0.05)
cbar.set_label("Maximum FRP", fontsize=12)

axes[0].text(0.05, 0.95, '(A)', transform=axes[0].transAxes, 
             fontsize=12, fontweight='bold', va='top', ha='right')
axes[1].text(0.05, 0.95, '(B)', transform=axes[1].transAxes, 
             fontsize=12, fontweight='bold', va='top', ha='right')
axes[2].text(0.05, 0.95, '(C)', transform=axes[2].transAxes, 
             fontsize=12, fontweight='bold', va='top', ha='right')

# Save the combined plot
plt.savefig(os.path.join(projdir, 'figures/Figure2_FRP_GriddingScheme.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Figure S2. Distribution of pixel ground area based on swath position.

In [ ]:
# Bin the sample positions into intervals
bin_edges = np.linspace(pix_area['sample'].min(), pix_area['sample'].max(), 20)  # Create 20 bins
pix_area['sample_bin'] = pd.cut(pix_area['sample'], bins=bin_edges, labels=bin_edges[:-1])

# Calculate mean pixel area for each bin
sample_summary = pix_area.groupby('sample_bin')['pix_area'].mean().reset_index()
sample_summary['sample_bin'] = sample_summary['sample_bin'].astype(float)  # Convert bin labels to float

# Create the figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(7.5, 2.5), constrained_layout=False)
fig.subplots_adjust(hspace=0.1, wspace=0.4)  

# ======= Left Plot: Histogram =======
sns.histplot(
    data=pix_area, 
    x="pix_area", 
    bins=50,  # Fine-grained bins for clarity
    color="#1f77b4", 
    kde=False, 
    ax=axes[0]
)

axes[0].set_xlabel("Pixel Area ($km^2$)", fontsize=9)
axes[0].set_ylabel("Frequency", fontsize=9)

# ======= Right Plot: Line Plot =======
axes[1].plot(
    sample_summary['sample_bin'], 
    sample_summary['pix_area'], 
    marker='o', color='orange', label="Mean Pixel Area"
)
axes[1].set_xlabel("Sample Position", fontsize=9)
axes[1].set_ylabel("Mean Pixel Area ($km^2$)", fontsize=9)
axes[0].tick_params(axis='both', which='major', labelsize=8)  # Change 10 to desired font size
axes[1].tick_params(axis='both', which='major', labelsize=8)

axes[1].grid(True, linestyle='--', alpha=0.6)

axes[0].text(-0.1, 1.1, "(A)", transform=axes[0].transAxes, fontsize=10, fontweight="bold", va="top", ha="right")
axes[1].text(-0.1, 1.1, "(B)", transform=axes[1].transAxes, fontsize=10, fontweight="bold", va="top", ha="right")

# Save the plot
out_plot = os.path.join(projdir, 'figures/FigureS1_PixelGroundArea_Distribution.png')
plt.savefig(out_plot, dpi=300, bbox_inches='tight')

# Show the plots
plt.show()